In [143]:
"""Implementing Different Layers to the NN"""

import tensorflow as tf
import numpy as np
import csv
import os
import random

from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
% matplotlib inline

# reset and start the graph
ops.reset_default_graph()

We are implementing different types of layers
1. Convolution Layer
2. Activation Layer
3. Max-Pool Layer
4. Fully connected Layer

We will generate two different data sets for this script, a 1-D data set (row of data) and a 2-D data set (similar to picture)

In [144]:
# ---------------------------------------------------|
# -------------------1D-data-------------------------|
# ---------------------------------------------------|


In [145]:
# tuning knobs
data_size = 25
conv_size = 5
maxpool_size = 5
stride_size = 1
num_outputs = 5

# ensure reproducibility
seed = 25
np.random.seed(seed)
tf.set_random_seed(seed)


In [146]:
# ----------Convolution function-----------
def conv_layer_1d(input_1d, my_filter, my_stride):
    """conv layer function"""
    # TensorFlow's 'conv2d()' function only works with 4D arrays:
    # [batch#, width, height, channels], we have 1 batch, and
    # 1 channel, but we do have width AND height this time.
    # So next we create the 4D array by inserting dimension 1's.

    """
    making everything as below
    input_1d ==> Tensor("Placeholder:0", shape=(25,), dtype=float32)
    input_2d ==> Tensor("ExpandDims:0", shape=(1, 25), dtype=float32)
    input_3d ==> Tensor("ExpandDims_1:0", shape=(1, 1, 25), dtype=float32)
    input_4d ==> Tensor("ExpandDims_2:0", shape=(1, 1, 25, 1), dtype=float32)"""
    # second argument points to the indices
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    # note down the stride difference
    conv_output = tf.nn.conv2d(input_4d, filter=my_filter, strides=[1, my_stride, my_stride, 1], padding='VALID')
    # get rid of unnecessary dimensions
    conv_output_1d = tf.squeeze(conv_output)
    return conv_output_1d


In [147]:
# ----------Activation function-----------
def activation(input_1d):
    return tf.nn.relu(input_1d)


In [148]:
# --------Max Pool--------
def max_pool(input_1d, width, my_stride):
    # Just like 'conv2d()' above, max_pool() works with 4D arrays.
    # [batch_size=1, width=1, height=num_input, channels=1]
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    # Perform the max pooling with strides = [1,1,1,1]
    # If we wanted to increase the stride on our data dimension, say by
    # a factor of '2', we put strides = [1, 1, 2, 1]
    # We will also need to specify the width of the max-window ('width')
    pool_output = tf.nn.max_pool(input_4d, ksize=[1, 1, width, 1], strides=[1, 1, my_stride, 1], padding='VALID')
    # get rid of extra dimensions
    pool_output_1d = tf.squeeze(pool_output)
    return pool_output_1d


In [149]:
# --------Fully connected--------
def fully_connected(input_layer, num_outputs):
    # First we find the needed shape of the multiplication weight matrix:
    # The dimension will be (length of input) by (num_outputs)
    weight_shape = tf.squeeze(tf.stack([tf.shape(input_layer), [num_outputs]]))
    # weight
    weight = tf.random_normal(weight_shape, stddev=.1)
    # bias
    bias = tf.random_normal(shape=[num_outputs])
    # Make the 1D input array into a 2D array for matrix multiplication
    input_layer_2d = tf.expand_dims(input_layer, 0)
    # perform matrix multiplication and add bias
    full_output = tf.add(bias, tf.matmul(input_layer_2d, weight))
    # get rid of unwanted dimenstions
    full_output_1d = tf.squeeze(full_output)
    return full_output_1d


In [150]:
data_1d = np.random.normal(size=data_size)

# placeholder
data_p_holder = tf.placeholder(tf.float32, shape=[data_size])

# filter for convolution
my_filter = tf.Variable(tf.random_normal(shape=[1, conv_size, 1, 1]))

# creating convolution layer
my_conv_output = conv_layer_1d(data_p_holder, my_filter, my_stride=stride_size)
activation_output = activation(my_conv_output)
max_pool_output = max_pool(activation_output, width=maxpool_size, my_stride=stride_size)
full_output = fully_connected(max_pool_output, num_outputs)

init_vars = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_vars)
    feed_dict = {data_p_holder: data_1d}
    print('>>>> 1D Data <<<<')
    """Output of conv_layer_1d"""
    print('Input = array of length %d' % (data_p_holder.shape.as_list()[0]))
    print('Convolution w/ filter, length = %d, stride size = %d, results in an array of length %d:' %
          (conv_size, stride_size, my_conv_output.shape.as_list()[0]))
    print('Convolution Layer output : \n', sess.run(my_conv_output, feed_dict=feed_dict))

    """Output of activation_output"""
    print('\nInput = above array of length %d' % (my_conv_output.shape.as_list()[0]))
    print('ReLU element wise returns an array of length %d:' % (activation_output.shape.as_list()[0]))
    print('Activation Layer output : \n', sess.run(activation_output, feed_dict=feed_dict))

    """Output of max_pool_output"""
    print('\nInput = above array of length %d' % (activation_output.shape.as_list()[0]))
    print('MaxPool, window length = %d, stride size = %d, results in the array of length %d' % (
    maxpool_size, stride_size, max_pool_output.shape.as_list()[0]))
    print('MaxPool Layer output : \n', sess.run(max_pool_output, feed_dict=feed_dict))

    """Output of full_output"""
    print('\nInput = above array of length %d' % (max_pool_output.shape.as_list()[0]))
    print('Fully connected layer on all 4 rows with %d outputs:' % (full_output.shape.as_list()[0]))
    print('FullyConnected Layer output : \n', sess.run(full_output, feed_dict=feed_dict))


>>>> 1D Data <<<<
Input = array of length 25
Convolution w/ filter, length = 5, stride size = 1, results in an array of length 21:
Convolution Layer output : 
 [ -3.13537598e-01   3.35255265e-03  -2.51171255e+00   2.69317269e+00
  -7.15947104e+00   4.47138786e+00  -2.31978464e+00  -9.98717844e-02
   3.64500523e+00  -5.84662914e+00   2.38776851e+00  -9.71207857e-01
   3.41508985e+00  -4.75846004e+00   2.65730858e+00  -2.02076888e+00
   4.84974289e+00  -7.25299644e+00   9.88106918e+00  -1.50379050e+00
  -3.74822474e+00]

Input = above array of length 21
ReLU element wise returns an array of length 21:
Activation Layer output : 
 [  0.00000000e+00   3.35255265e-03   0.00000000e+00   2.69317269e+00
   0.00000000e+00   4.47138786e+00   0.00000000e+00   0.00000000e+00
   3.64500523e+00   0.00000000e+00   2.38776851e+00   0.00000000e+00
   3.41508985e+00   0.00000000e+00   2.65730858e+00   0.00000000e+00
   4.84974289e+00   0.00000000e+00   9.88106918e+00   0.00000000e+00
   0.00000000e+00]



In [151]:
# ---------------------------------------------------|
# -------------------2D-data-------------------------|
# ---------------------------------------------------|

In [152]:
row_size = 10
col_size = 10
conv_size = 2
conv_stride_size = 2
max_pool_size = 2
max_pool_stride_size = 1


# --------Convolution--------
def conv_2d_layer(input_2d, my_filter, stride_size):
    # TensorFlow's 'conv2d()' function only works with 4D arrays:
    # [batch#, width, height, channels], we have 1 batch, and
    # 1 channel, but we do have width AND height this time.
    # So next we create the 4D array by inserting dimension 1's.
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    # Note the stride difference below!
    convolution_output = tf.nn.conv2d(input_4d, my_filter, strides=[1, stride_size, stride_size, 1], padding='VALID')
    # get rid or unnecessary dimensions
    convolution_2d_output = tf.squeeze(convolution_output)
    return convolution_2d_output


# --------Activation--------
def activation_layer(input_2d):
    return tf.nn.relu(input_2d)


# --------Max Pool--------
def max_pool(input_2d, width, height, stride):
    # Just like 'conv2d()' above, max_pool() works with 4D arrays.
    # [batch_size=1, width=given, height=given, channels=1]
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    # Perform the max pooling with strides = [1,1,1,1]
    # If we wanted to increase the stride on our data dimension, say by
    # a factor of '2', we put strides = [1, 2, 2, 1]
    pool_output = tf.nn.max_pool(input_4d, ksize=[1, height, width, 1],
                                 strides=[1, stride, stride, 1],
                                 padding='VALID')
    # Get rid of unnecessary dimensions
    pool_output_2d = tf.squeeze(pool_output)
    return pool_output_2d


# --------Fully Connected--------
def fully_connected(input_layer, num_outputs):
    # In order to connect our whole W byH 2d array, we first flatten it out to
    # a W times H 1D array.
    flat_input = tf.reshape(input_layer, [-1])  # making 1D array
    # We then find out how long it is, and create an array for the shape of
    # the multiplication weight = (WxH) by (num_outputs)
    weight_shape = tf.squeeze(tf.stack([tf.shape(flat_input), [num_outputs]]))
    # Initialize the weight
    weight = tf.random_normal(weight_shape, stddev=0.1)
    # Initialize the bias
    bias = tf.random_normal(shape=[num_outputs])
    # Now make the flat 1D array into a 2D array for multiplication
    input_2d = tf.expand_dims(flat_input, 0)
    # Multiply and add the bias
    full_output = tf.add(tf.matmul(input_2d, weight), bias)
    # Get rid of extra dimension
    full_output_2d = tf.squeeze(full_output)
    return full_output_2d


# ensure reproducibility
seed = 13
np.random.seed(seed)
tf.set_random_seed(seed)

# Generate 2D data
data_size = [row_size, col_size]
data_2d = np.random.normal(size=data_size)

# placeholder
data_2d_place_holder = tf.placeholder(dtype=tf.float32, shape=data_size)

# convolution filter
my_filter = tf.Variable(tf.random_normal(shape=[conv_size, conv_size, 1, 1]))

# convolution layer
my_conv_output = conv_2d_layer(data_2d_place_holder, my_filter, stride_size=conv_stride_size)

# create activation layer
my_activation_output = activation_layer(my_conv_output)

# create maxpool layer
my_maxpool_output = max_pool(my_activation_output, max_pool_size, max_pool_size, stride=max_pool_stride_size)

# Create Fully Connected Layer
my_full_output = fully_connected(my_maxpool_output, num_outputs)

# init vars
init_vars = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_vars)
    feed_dict = {data_2d_place_holder: data_2d}

    print('>>>> 2D Data <<<<')

    # Convolution Output
    print('Input = %s array' % (data_2d_place_holder.shape.as_list()))
    print('%s Convolution, stride size = [%d, %d] , results in the %s array' %
          (my_filter.get_shape().as_list()[:2], conv_stride_size, conv_stride_size,
           my_conv_output.shape.as_list()))
    print(sess.run(my_conv_output, feed_dict=feed_dict))

    # Activation Output
    print('\nInput = the above %s array' % (my_conv_output.shape.as_list()))
    print('ReLU element wise returns the %s array' % (my_activation_output.shape.as_list()))
    print(sess.run(my_activation_output, feed_dict=feed_dict))

    # Max Pool Output
    print('\nInput = the above %s array' % (my_activation_output.shape.as_list()))
    print('MaxPool, stride size = [%d, %d], results in %s array' %
          (max_pool_stride_size, max_pool_stride_size, my_maxpool_output.shape.as_list()))
    print(sess.run(my_maxpool_output, feed_dict=feed_dict))

    # Fully Connected Output
    print('\nInput = the above %s array' % (my_maxpool_output.shape.as_list()))
    print('Fully connected layer on all %d rows results in %s outputs:' %
          (my_maxpool_output.shape.as_list()[0], my_full_output.shape.as_list()[0]))
    print(sess.run(my_full_output, feed_dict=feed_dict))


>>>> 2D Data <<<<
Input = [10, 10] array
[2, 2] Convolution, stride size = [2, 2] , results in the [5, 5] array
[[-0.53970814 -1.37039399  0.16790187  0.80627304  2.52373457]
 [-3.81318855  3.66644287 -0.88086319 -2.76224279  5.07420492]
 [-2.33438158  0.5242815   0.71433663 -1.56049573 -1.59068143]
 [ 1.18478489 -1.71592748  1.03441024  1.0885272  -0.54076439]
 [ 5.41466427 -4.74507046 -0.94251752 -4.45919561  2.67982888]]

Input = the above [5, 5] array
ReLU element wise returns the [5, 5] array
[[ 0.          0.          0.16790187  0.80627304  2.52373457]
 [ 0.          3.66644287  0.          0.          5.07420492]
 [ 0.          0.5242815   0.71433663  0.          0.        ]
 [ 1.18478489  0.          1.03441024  1.0885272   0.        ]
 [ 5.41466427  0.          0.          0.          2.67982888]]

Input = the above [5, 5] array
MaxPool, stride size = [1, 1], results in [4, 4] array
[[ 3.66644287  3.66644287  0.80627304  5.07420492]
 [ 3.66644287  3.66644287  0.71433663  5.07